Load packages

In [13]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Pkg
Pkg.instantiate()
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

purging the data model entirely - empty the schema

In [15]:
if (haskey(ENV, "GITPOD_REPO_ROOT"))
    run(```psql -f sqlsnippets/droptables.sql```)
else
    println("execute this in Your REPL")
    println("""run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)
        
        """)
end


execute this in Your REPL
run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)




Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [16]:
LifeInsuranceDataModel.load_model()

cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


┌ Info: CREATE TABLE schema_migrations (version varchar(30))
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: Created table schema_migrations
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:305
┌ Info: CREATE TABLE histories (id BIGSERIAL PRIMARY KEY, dummy INTEGER )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE versions (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE workflows (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, tsw_validfrom TIMESTAMPTZ (3), tsdb_validfrom TIMESTAMPTZ (3), is_committe

┌ Info: CREATE TABLE testdummySubComponentRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES testdummySubComponents(id), ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 9223372036854775807 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE EXTENSION IF NOT EXISTS btree_gist;
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  extension "btree_gist" already exists, skipping
┌ Info: ALTER TABLE validityIntervals 
│ ADD CONSTRAINT bitemp EXCLUDE USING GIST (ref_version WITH =, is_committed WITH =, tsrworld WITH &&, tsrdb WITH &&)
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE OR REPLACE FUNCTION f_bitempranges ()
│

┌ Info: CREATE TABLE TarifFItemRoles (id BIGSERIAL PRIMARY KEY, domain VARCHAR , value VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE ProductPartRoles (id BIGSERIAL PRIMARY KEY, domain VARCHAR , value VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE productParts (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_super BIGINT REFERENCES products(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE productPartRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES productParts(id) ON DELETE CASCADE, position BIGINT , ref_role BIGINT REFERENCES productpartroles(id) ON DELETE CASCADE, r

┌ Info: CREATE TRIGGER pr_versions_trig
│ BEFORE INSERT OR UPDATE ON partnerRevisions
│ FOR EACH ROW EXECUTE PROCEDURE f_versionrange();
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: ALTER TABLE partnerRevisions
│ ADD CONSTRAINT pr_versionrange EXCLUDE USING GIST (ref_component WITH =, ref_valid WITH &&)
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE contractPartnerRoles (id BIGSERIAL PRIMARY KEY, domain VARCHAR , value VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE contractPartnerRefs (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_super BIGINT REFERENCES contracts(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/.juli

┌ Info: CREATE TABLE TariffItemRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES tariffitems(id) ON DELETE CASCADE, ref_role BIGINT REFERENCES tariffitemroles(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_valid INT8RANGE , description VARCHAR , ref_tariff BIGINT REFERENCES tariffs(id) ON DELETE CASCADE, net_premium FLOAT , annuity_immediate FLOAT , deferment INTEGER , annuity_due FLOAT )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TRIGGER pitrr_versions_trig
│ BEFORE INSERT OR UPDATE ON TariffItemRevisions
│ FOR EACH ROW EXECUTE PROCEDURE f_versionrange();
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: ALTER TABLE TariffItemRevisions
│ ADD CONSTRAINT pitrr_versionrange EXCLUDE USING GIST (ref_component WI

┌ Info: SELECT "contractpartnerroles"."id" AS "contractpartnerroles_id", "contractpartnerroles"."domain" AS "contractpartnerroles_domain", "contractpartnerroles"."value" AS "contractpartnerroles_value" FROM "contractpartnerroles" ORDER BY contractpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffitempartnerroles"."id" AS "tariffitempartnerroles_id", "tariffitempartnerroles"."domain" AS "tariffitempartnerroles_domain", "tariffitempartnerroles"."value" AS "tariffitempartnerroles_value" FROM "tariffitempartnerroles" ORDER BY tariffitempartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" ORDER BY tariffitemroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


4-element Vector{Int64}:
 1
 2
 3
 4

Or just connect to an existing model

In [17]:
LifeInsuranceDataModel.connect()

PostgreSQL connection (CONNECTION_OK) with parameters:
  user = postgres
  password = ********************
  channel_binding = prefer
  dbname = postgres
  host = 127.0.0.1
  port = 5432
  client_encoding = UTF8
  options = -c DateStyle=ISO,YMD -c IntervalStyle=iso_8601 -c TimeZone=UTC
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 0
  sslsni = 1
  ssl_min_protocol_version = TLSv1.2
  gssencmode = prefer
  krbsrvname = postgres
  target_session_attrs = any

Create a Partner

In [18]:
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

Partner1 = p.id.value

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 1 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 1 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Searc

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


1

Create tariffs

In [19]:
# create Tariffs
function create_tariff(dsc,mt)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc,mortality_table=mt)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
    t.id.value
end

LifeRiskTariff = create_tariff("Life Risk Insurance", "1980 CET - Male Nonsmoker, ANB")
TerminalIllnessTariff = create_tariff("Terminal Illness", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
OccupationalDisabilityTariff = create_tariff("Occupational Disability", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
ProfitParticipationTariff = create_tariff("Profit participation", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
LifeRiskTariff2 = create_tariff("Life Risk Insurance", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")

find(TariffRevision)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 2 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'2' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 2 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Searc

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 3 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'3' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" W

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 4 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'4' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" W

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'4' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.juli

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'5' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.juli

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'6' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.juli

5-element Vector{TariffRevision}:
 TariffRevision
| KEY                     | VALUE                          |
|-------------------------|--------------------------------|
| description::String     | Life Risk Insurance            |
| id::DbId                | 1                              |
| mortality_table::String | 1980 CET - Male Nonsmoker, ANB |
| ref_component::DbId     | 1                              |
| ref_invalidfrom::DbId   | 9223372036854775807            |
| ref_validfrom::DbId     | 2                              |

 TariffRevision
| KEY                     | VALUE                                                                |
|-------------------------|----------------------------------------------------------------------|
| description::String     | Terminal Illness                                                     |
| id::DbId                | 2                                                                    |
| mortality_table::String | 2001 VBT Residual Sta

In [20]:
find(Tariff,SQLWhereExpression("id=?",ProfitParticipationTariff))
find(Tariff, SQLWhereExpression("id=?", TerminalIllnessTariff))

┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" WHERE id=4 ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" WHERE id=2 ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


1-element Vector{Tariff}:
 Tariff
| KEY               | VALUE |
|-------------------|-------|
| id::DbId          | 2     |
| ref_history::DbId | 3     |
| ref_version::DbId | 3     |


Create Product

In [21]:
p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Lif Risk")

w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

LifeRiskProduct = p.id.value
println(LifeRiskProduct)

p = Product()
pr = ProductRevision(description="Life Risk - Terminal Illness")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Life Risk")

pp3 = ProductPart()
ppr3 = ProductPartRevision(ref_tariff=TerminalIllnessTariff, ref_role=ppRole["Supplementary Coverage - Terminal Illness"], description="additional cover Terminal Illness")

pp4 = ProductPart()
ppr4 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Terminal Illness")

pp5 = ProductPart()
ppr5 = ProductPartRevision(ref_tariff=OccupationalDisabilityTariff, ref_role=ppRole["Supplementary Coverage - Occupational Disablity"], description="additional cover Occupational Disablity")

pp6 = ProductPart()
ppr6 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Occ.Disablity")



w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
create_subcomponent!(p, pp3, ppr3, w0)
create_subcomponent!(p, pp4, ppr4, w0)
create_subcomponent!(p, pp5, ppr5, w0)
create_subcomponent!(p, pp6, ppr6, w0)
commit_workflow!(w0)

LifeRiskTIODProduct = p.id.value
println(LifeRiskTIODProduct)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 7 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'7' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 7 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Searc

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'7' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2014-05-31T20:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: UPDATE validityintervals

1


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO productparts ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'8', E'9223372036854775807', E'2' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "productparts"."id" AS "productparts_id", "productparts"."ref_history" AS "productparts_ref_history", "productparts"."ref_version" AS "productparts_ref_version", "productparts"."ref_super" AS "productparts_ref_super" FROM "productparts" WHERE "id" = 4 ORDER BY productparts.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO productpartrevisions ( "ref_component", "ref_validfrom", "ref_invalid

2


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO productparts ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'8', E'9223372036854775807', E'2' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "productparts"."id" AS "productparts_id", "productparts"."ref_history" AS "productparts_ref_history", "productparts"."ref_version" AS "productparts_ref_version", "productparts"."ref_super" AS "productparts_ref_super" FROM "productparts" WHERE "id" = 8 ORDER BY productparts.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO productpartrevisions ( "ref_component", "ref_validfrom", "ref_invalid

Create contract

In [22]:
    w1 = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    
    create_entity!(w1)
    c = Contract()
    cr = ContractRevision(description="contract creation properties")
    create_component!(c, cr, w1)
    
    cpr = ContractPartnerRef(ref_super=c.id)
    cprr = ContractPartnerRefRevision(ref_partner=Partner1, ref_role=cpRole["Policy Holder"], description="policiyholder ref properties")
    create_subcomponent!(c, cpr, cprr, w1)
    # pi 1
    cpi = ProductItem(ref_super=c.id)
    cpir = ProductItemRevision(position=1, ref_product=LifeRiskTIODProduct, description="from contract creation")
    create_subcomponent!(c, cpi, cpir, w1)
    # pi 1 ti 1 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=LifeRiskTariff, ref_role=titrRole["Main Coverage - Life"], description="Life Risk tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 1 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 2 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 2 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 3 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=TerminalIllnessTariff, ref_role=titrRole["Supplementary Coverage - Terminal Illness"], description="Terminal Illness tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 3 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 4 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profitparticipation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 4 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    
    # pi 1 ti 5 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Supplementary Coverage - Occupational Disablity"], description="Occupational Disablity tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 5 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 6 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit Participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 6 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    
    
    
    commit_workflow!(w1)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 9 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 9 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Searc

┌ Info: INSERT INTO tariffitemrevisions ( "ref_component", "ref_role", "ref_validfrom", "ref_invalidfrom", "description", "ref_tariff", "net_premium", "annuity_immediate", "deferment", "annuity_due" ) VALUES ( E'1', E'1', E'9', E'9223372036854775807', E'Life Risk tariff parameters', E'1', 0.0, 0.0, 0, 0.0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (1,1,1,9,9223372036854775807,,"Life Risk tariff parameters",1,0,0,0,0)
┌ Info: SELECT "tariffitemrevisions"."id" AS "tariffitemrevisions_id", "tariffitemrevisions"."ref_component" AS "tariffitemrevisions_ref_component", "tariffitemrevisions"."ref_role" AS "tariffitemrevisions_ref_role", "tariffitemrevisions"."ref_validfrom" AS "tariffitemrevisions_ref_validfrom", "tariffitemrevisions"."ref_invalidfrom" AS "tariffitemrevisions_ref_invalidfrom", "tariffitemrevisions"."description" AS "tariffitemrevisions_description", "tariffitemrevisions"."ref_tar

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO tariffitems ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'9', E'9223372036854775807', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffitems"."id" AS "tariffitems_id", "tariffitems"."ref_history" AS "tariffitems_ref_history", "tariffitems"."ref_version" AS "tariffitems_ref_version", "tariffitems"."ref_super" AS "tariffitems_ref_super" FROM "tariffitems" WHERE "id" = 2 ORDER BY tariffitems.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO tariffitemrevisions ( "ref_component", "ref_role", "ref_validfrom", "ref_invalid

┌ Info: INSERT INTO tariffitemrevisions ( "ref_component", "ref_role", "ref_validfrom", "ref_invalidfrom", "description", "ref_tariff", "net_premium", "annuity_immediate", "deferment", "annuity_due" ) VALUES ( E'4', E'4', E'9', E'9223372036854775807', E'Profitparticipation tariff parameters', E'4', 0.0, 0.0, 0, 0.0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (4,4,4,9,9223372036854775807,,"Profitparticipation tariff parameters",4,0,0,0,0)
┌ Info: SELECT "tariffitemrevisions"."id" AS "tariffitemrevisions_id", "tariffitemrevisions"."ref_component" AS "tariffitemrevisions_ref_component", "tariffitemrevisions"."ref_role" AS "tariffitemrevisions_ref_role", "tariffitemrevisions"."ref_validfrom" AS "tariffitemrevisions_ref_validfrom", "tariffitemrevisions"."ref_invalidfrom" AS "tariffitemrevisions_ref_invalidfrom", "tariffitemrevisions"."description" AS "tariffitemrevisions_description", "tariffite

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO tariffitempartnerrefs ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'9', E'9223372036854775807', E'6' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffitempartnerrefs"."id" AS "tariffitempartnerrefs_id", "tariffitempartnerrefs"."ref_history" AS "tariffitempartnerrefs_ref_history", "tariffitempartnerrefs"."ref_version" AS "tariffitempartnerrefs_ref_version", "tariffitempartnerrefs"."ref_super" AS "tariffitempartnerrefs_ref_super" FROM "tariffitempartnerrefs" WHERE "id" = 6 ORDER BY tariffitempartnerrefs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreS

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'9' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2014-05-31T20:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: UPDATE validityintervals

update Contract yellow

In [23]:
cr1 = ContractRevision(ref_component=c.id, description="contract 1, 2nd mutation")
w2 = Workflow(
    ref_history=w1.ref_history,
    tsw_validfrom=ZonedDateTime(2016, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w2)
update_component!(cr, cr1, w2)
commit_workflow!(w2)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 10 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'9', E'10', E'2038-01-19T03:14:06.999+00:00', E'2016-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "wor

┌ Info: UPDATE validityintervals SET  "id" = E'10', "ref_history" = E'9', "ref_version" = E'10', "tsdb_validfrom" = E'2022-07-25T16:21:26.261+01:00', "tsworld_validfrom" = E'2016-05-30T20:00:01.001+00:00', "tsdb_invalidfrom" = E'2038-01-19T03:14:06.999+00:00', "tsworld_invalidfrom" = E'2038-01-19T03:14:06.999+00:00', "is_committed" = 1 WHERE validityintervals.id = '10' RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (10,9,10,"2016-05-30 20:00:01.001+00","2038-01-19 03:14:06.999+00","2022-07-25 15:21:26.261+00","2038-01-19 03:14:06.999+00","[""2016-05-30 20:00:01.001+00"",""2038-01-19 03:14:06.999+00"")","[""2022-07-25 15:21:26.249+00"",""2038-01-19 03:14:06.999+00"")",1)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."t

   update Contract red

In [24]:

cr2 = ContractRevision(ref_component=c.id, description="contract 1, 3rd mutation retrospective")
w3 = Workflow(
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2015, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w3)
update_component!(cr1, cr2, w3)


commit_workflow!(w3)


┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 11 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'9', E'11', E'2038-01-19T03:14:06.999+00:00', E'2015-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "wor

┌ Info: INSERT INTO validityintervals ( "ref_history", "ref_version", "tsdb_validfrom", "tsworld_validfrom", "tsdb_invalidfrom", "tsworld_invalidfrom", "is_committed" ) VALUES ( E'9', E'9', E'2022-07-25T16:21:26.308+01:00', E'2014-05-30T20:00:01.001+00:00', E'2038-01-19T03:14:06.999+00:00', E'2015-05-30T20:00:01.001+00:00', 1 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (13,9,9,"2014-05-30 20:00:01.001+00","2015-05-30 20:00:01.001+00","2022-07-25 15:21:26.308+00","2038-01-19 03:14:06.999+00",,,1)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom

┌ Info: UPDATE validityintervals SET  "id" = E'12', "ref_history" = E'9', "ref_version" = E'11', "tsdb_validfrom" = E'2022-07-25T16:21:26.308+01:00', "tsworld_validfrom" = E'2015-05-30T20:00:01.001+00:00', "tsdb_invalidfrom" = E'2038-01-19T03:14:06.999+00:00', "tsworld_invalidfrom" = E'2038-01-19T03:14:06.999+00:00', "is_committed" = 1 WHERE validityintervals.id = '12' RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (12,9,11,"2015-05-30 20:00:01.001+00","2038-01-19 03:14:06.999+00","2022-07-25 15:21:26.308+00","2038-01-19 03:14:06.999+00","[""2015-05-30 20:00:01.001+00"",""2038-01-19 03:14:06.999+00"")","[""2022-07-25 15:21:26.297+00"",""2038-01-19 03:14:06.999+00"")",1)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."t

In [25]:
w4 = Workflow(
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2018, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w4)
cr3 = ContractRevision(ref_component=c.id, description="contract 1, 4th mutation")
update_component!(cr2, cr3, w4)

# pi 2
LifeRiskTIODProduct = find(Product, SQLWhereExpression("id=?", 2))[1].id.value
Partner1
PartnerRole = tiprRole["Insured Person"]

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(position=2, ref_product=LifeRiskTIODProduct, description="from contract 4th mutation")
create_subcomponent!(c, cpi, cpir, w4)

println("vor neuem Teil")



map(find(ProductPart, SQLWhereExpression("ref_super=?", LifeRiskTIODProduct))) do pp
    println(pp.id.value)
    map(find(ProductPartRevision, SQLWhereExpression("ref_component=?", pp.id.value))) do ppr
        println(ppr.description)
        ti=TariffItem(ref_super=cpi.id)
        tir = TariffItemRevision(ref_role=ppr.ref_role, ref_tariff=ppr.ref_tariff, description=ppr.description)
        create_subcomponent!(cpi, ti, tir, w4)
        tip = TariffItemPartnerRef(ref_super=cpi.id)
        tipr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=PartnerRole)
        create_subcomponent!(cpi, tip, tipr, w4)
        println(tir)
        println(tipr)
    end
end
println("vor rollback")
# rollback_workflow!(w4)
commit_workflow!(w4)

vor neuem Teil


┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 12 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'9', E'12', E'2038-01-19T03:14:06.999+00:00', E'2018-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "wor

3


┌ Info: SELECT "productparts"."id" AS "productparts_id", "productparts"."ref_history" AS "productparts_ref_history", "productparts"."ref_version" AS "productparts_ref_version", "productparts"."ref_super" AS "productparts_ref_super" FROM "productparts" WHERE ref_super=2 ORDER BY productparts.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=3 ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


...

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=6 ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSE

4
Profit participation Life Risk
TariffItemRevision(ref_component::DbId=8,ref_role::DbId=4,ref_tariff::DbId=4,annuity_due::Float64=0.0,ref_validfrom::DbId=12,net_premium::Float64=0.0,annuity_immediate::Float64=0.0,description::String=Profit participation Life Risk,id::DbId=8,deferment::Int64=0,ref_invalidfrom::DbId=9223372036854775807)
TariffItemPartnerRefRevision(ref_validfrom::DbId=12,ref_invalidfrom::DbId=9223372036854775807,id::DbId=8,ref_partner::DbId=1,ref_component::DbId=8,ref_role::DbId=1,description::String=)
5
additional cover Terminal Illness
TariffItemRevision(ref_component::DbId=9,ref_role::DbId=3,ref_tariff::DbId=2,annuity_due::Float64=0.0,ref_validfrom::DbId=12,net_premium::Float64=0.0,annuity_immediate::Float64=0.0,description::String=additional cover Terminal Illness,id::DbId=9,deferment::Int64=0,ref_invalidfrom::DbId=9223372036854775807)
TariffItemPartnerRefRevision(ref_validfrom::DbId=12,ref_invalidfrom::DbId=9223372036854775807,id::DbId=9,ref_partner::DbId=1,ref_com

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


TariffItemRevision(ref_component::DbId=11,ref_role::DbId=2,ref_tariff::DbId=3,annuity_due::Float64=0.0,ref_validfrom::DbId=12,net_premium::Float64=0.0,annuity_immediate::Float64=0.0,description::String=additional cover Occupational Disablity,id::DbId=11,deferment::Int64=0,ref_invalidfrom::DbId=9223372036854775807)
TariffItemPartnerRefRevision(ref_validfrom::DbId=12,ref_invalidfrom::DbId=9223372036854775807,id::DbId=11,ref_partner::DbId=1,ref_component::DbId=11,ref_role::DbId=1,description::String=)
8
Profit participation Occ.Disablity
TariffItemRevision(ref_component::DbId=12,ref_role::DbId=4,ref_tariff::DbId=4,annuity_due::Float64=0.0,ref_validfrom::DbId=12,net_premium::Float64=0.0,annuity_immediate::Float64=0.0,description::String=Profit participation Occ.Disablity,id::DbId=12,deferment::Int64=0,ref_invalidfrom::DbId=9223372036854775807)
TariffItemPartnerRefRevision(ref_validfrom::DbId=12,ref_invalidfrom::DbId=9223372036854775807,id::DbId=12,ref_partner::DbId=1,ref_component::DbId=12

┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=8 ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO tariffitems ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'9', E'9223372036854775807', E'2' ) RETURNING id
└ 